In [24]:
# Filter out features that do not fall in any land use/land cover class

# Import the Earth Engine Python Package
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Import the FeatureCollection
fc_path = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/soil_organic_carbon/26-11-lulc'
fc_original = ee.FeatureCollection(fc_path)

# Print the number of features in the FeatureCollection
print(fc_original.size().getInfo())

# Print the columns of the FeatureCollection
print(fc_original.first().propertyNames().getInfo())

# Print the minimum and maximum values of the 'natural' property
print(fc_original.aggregate_min('natural').getInfo())
print(fc_original.aggregate_max('natural').getInfo())

12663
['elevation', 'natural', 'year', 'dataset_id', 'restingas', 'pastagem', 'estoque', 'antropico', 'formacaoSavanica', 'mosaicoDeUsos', 'formacaoCampestre', 'formacaoFlorestal', 'silvicultura', 'Area_Estavel', 'outrasFormacoesFlorestais', 'IFN_index', 'lavouras', 'campoAlagadoAreaPantanosa', 'system:index']
0
58


In [25]:
# Identify samples with land use/land cover classes

# Consider the following properties of the FeatureCollection:
# 'campoAlagadoAreaPantanosa', 'formacaoCampestre', 'formacaoFlorestal',
# 'formacaoSavanica', 'lavouras', 'mosaicoDeUsos', 'outrasFormacoesFlorestais',
# 'pastagem', 'restingas', 'silvicultura'
# Create a columns that contains the sum of the values of these properties
# and print the result.

# Create a list of the properties
properties = ['campoAlagadoAreaPantanosa', 'formacaoCampestre', 'formacaoFlorestal',
              'formacaoSavanica', 'lavouras', 'mosaicoDeUsos', 'outrasFormacoesFlorestais',
              'pastagem', 'restingas', 'silvicultura']

# Create a column that contains the sum of the values of the properties
fc_original = fc_original.map(lambda feature: feature.set('sum', ee.Number(0)))
for property in properties:
    fc_original = fc_original.map(lambda feature: feature.set('sum', ee.Number(feature.get('sum')).add(feature.get(property))))
                                  
# Print the minimum and maximum values of the 'sum' property
print(fc_original.aggregate_min('sum').getInfo())
print(fc_original.aggregate_max('sum').getInfo())

0
58


In [26]:
# Drop a feature if the sum of the values of the properties is equal to zero
# and print the result.
fc_filter = fc_original.filter(ee.Filter.neq('sum', 0))
print(fc_filter.size().getInfo())

12289


In [27]:
# Replicate samples from natural vegetation classes that are stable over time

# Create a columns with the sum of the values of the following properties:
# 'formacaoCampestre', 'formacaoFlorestal', 'formacaoSavanica', 'outrasFormacoesFlorestais',
# 'restingas', 'campoAlagadoAreaPantanosa'.
# Print the result.
# Create a list of the properties
properties = ['campoAlagadoAreaPantanosa', 'formacaoCampestre', 'formacaoFlorestal',
              'formacaoSavanica', 'outrasFormacoesFlorestais', 'restingas']

# Create a column that contains the sum of the values of the properties
fc_filter = fc_filter.map(lambda feature: feature.set('sum', ee.Number(0)))
for property in properties:
    fc_filter = fc_filter.map(lambda feature: feature.set('sum', ee.Number(feature.get('sum')).add(feature.get(property))))

# Print the minimum and maximum values of the 'sum' property
print(fc_filter.aggregate_min('sum').getInfo()) # zero means antropized area
print(fc_filter.aggregate_max('sum').getInfo())

0
58


In [28]:
# Identify the features for which natural > 0 and year > 2004 and sum of the values of the properties is greater than 39
# and print the result.
fc_stable = fc_filter.filter(ee.Filter.gt('sum', 39)).filter(ee.Filter.gt('year', 2004)).filter(ee.Filter.gt('natural', 0))
print(fc_stable.size().getInfo())

1686


In [29]:
# Print the minimum and maximum values of the 'year' property
print(fc_stable.aggregate_min('year').getInfo())
print(fc_stable.aggregate_max('year').getInfo())

2005
2023


In [30]:
# Print the minimum and maximum values of the 'natural' property
print(fc_stable.aggregate_min('natural').getInfo())
print(fc_stable.aggregate_max('natural').getInfo())

40
58


In [31]:
# Print the minimum and maximum values of the 'antropico' property
print(fc_stable.aggregate_min('antropico').getInfo())
print(fc_stable.aggregate_max('antropico').getInfo())

0
0


In [32]:
# Subtract 10 from the 'year' property
fc_stable_10 = fc_stable.map(lambda feature: feature.set('year', ee.Number(feature.get('year')).subtract(10)))

# Subtract 20 from the 'year' property
fc_stable_20 = fc_stable.map(lambda feature: feature.set('year', ee.Number(feature.get('year')).subtract(20)))
							 
# Stack the three FeatureCollections: fc_filter, fc_stable_10, fc_stable_20
fc_stacked = fc_filter.merge(fc_stable_10).merge(fc_stable_20)

# Print the number of features in the stacked FeatureCollection
print(fc_stacked.size().getInfo())

15661


In [33]:
# Row order the stacked FeatureCollection by the 'dataset_id' property
fc_stacked = fc_stacked.sort('dataset_id')

# Print the 'dataset_id' and 'year' properties of the first 10 features in the stacked
# FeatureCollection
print(fc_stacked.limit(10).aggregate_array('dataset_id').getInfo())
print(fc_stacked.limit(10).aggregate_array('year').getInfo())

['ctb0003-sm-dnos-001', 'ctb0003-sm-dnos-002', 'ctb0003-sm-dnos-003', 'ctb0003-sm-dnos-004', 'ctb0003-sm-dnos-005', 'ctb0003-sm-dnos-005', 'ctb0003-sm-dnos-005', 'ctb0003-sm-dnos-006', 'ctb0003-sm-dnos-007', 'ctb0003-sm-dnos-007']
[2009, 2009, 2009, 2009, 2009, 1999, 1989, 2009, 2009, 1999]


In [34]:
# Print the minimum and maximum values of the 'year' property
print(fc_stacked.aggregate_min('year').getInfo())
print(fc_stacked.aggregate_max('year').getInfo())

1948
2023


In [36]:
# Save the FeatureCollection to a new asset
fc_path = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/ORIGINAIS/2024-12-01-organic-carbon-stock-gram-per-square-meter-filter-rep'
# Keep only the following columns: ['dataset_id', 'year', 'estoque', 'IFN_index']
fc_stacked = fc_stacked.select(['dataset_id', 'year', 'estoque', 'IFN_index'])
# Export the FeatureCollection
task = ee.batch.Export.table.toAsset(collection=fc_stacked, description='fc_stacked', assetId=fc_path)
task.start()
print(task.status())

{'state': 'READY', 'description': 'fc_stacked', 'priority': 100, 'creation_timestamp_ms': 1733108519556, 'update_timestamp_ms': 1733108519556, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': '3Y4G4BTFSXJRIR7B72KYQKOR', 'name': 'projects/846873190609/operations/3Y4G4BTFSXJRIR7B72KYQKOR'}


In [41]:
# Plot a map of the FeatureCollection fc_original, fc_filter (blue), and fc_stable_10 (red)

# Import the Folium library.
import folium

# Define a map centered on the Amazon rainforest.
map = folium.Map(location=[-2.5, -55], zoom_start=5)

# Add the FeatureCollection fc_original to the map.
map.add_ee_layer(fc_original, {}, 'Original')

# Add the FeatureCollection fc_filter to the map.
map.add_ee_layer(fc_filter, {'color': 'blue'}, 'Filter')

# Add the FeatureCollection fc_stable_10 to the map.
map.add_ee_layer(fc_stable_10, {'color': 'red'}, 'Stable')

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Display the map.
display(map)